## XGBoost

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import numpy as np

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings("ignore")


### Importing the data

In [5]:
# Loading the data
FC_data = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed FC/Merged_FC.csv", index_col = 0)
FC_data.index.name = "Subject ID"
FC_data.head()

FC_pheno_data = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed FC matrix with Pheno/FC_Merged.csv", index_col = 0)
FC_pheno_data.index.name = "Subject ID"
FC_pheno_data.head()

FC_PCA = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed FC matrix with PCA extracted data/PCA_feature.csv", index_col = 0)
FC_PCA.index.name = "Subject ID"
FC_PCA.head()

Reho = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed ReHo/All_ReHo.csv", index_col = 0)
Reho.index.name = "Subject ID"
Reho.head()

falff = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed fALFF/All_falff.csv", index_col = 0)
falff.index.name = "Subject ID"
falff.head()

,DX,0,1,2,3,4,5,6,7,8,...,17945,17946,17947,17948,17949,17950,17951,17952,17953,17954
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0,0.082767,-0.202121,-0.253291,0.143162,-0.212533,0.572503,-0.346531,0.010921,0.014705,...,0.112687,0.171248,-0.073971,-0.413591,0.238855,0.300857,0.177778,0.658069,-0.052805,0.145006
1019436,3,0.216872,-0.055456,0.274632,0.057173,0.318357,0.334924,-0.285290,-0.093749,0.051842,...,-0.466976,-0.189028,0.048556,-0.476408,0.064047,-0.008339,0.424513,0.450524,-0.022595,0.231871
1043241,0,-0.060757,0.218841,-0.220541,-0.009787,-0.018797,0.102055,-0.207456,-0.332605,0.157679,...,-0.118490,-0.123154,-0.287799,-0.404065,0.111559,-0.233413,-0.120683,0.083335,-0.058685,0.343333
1266183,0,-0.063801,0.061519,-0.011792,0.016329,0.135984,0.164675,0.174189,-0.108558,-0.031854,...,-0.256891,-0.414658,-0.549495,0.543498,0.292311,0.164908,-0.031181,0.084472,-0.235999,-0.609832
1535233,0,0.022708,0.380467,0.404897,0.422225,0.546707,-0.022366,0.218834,0.274712,0.191838,...,-0.238173,-0.059068,-0.020537,-0.465405,0.143333,0.194637,0.410916,0.510831,-0.106011,0.137268


,DX,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status,0,1,...,17945,17946,17947,17948,17949,17950,17951,17952,17953,17954
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0,44.0,47.0,44.0,99.0,115.0,103.0,1,0.082767,-0.202121,...,0.112687,0.171248,-0.073971,-0.413591,0.238855,0.300857,0.177778,0.658069,-0.052805,0.145006
1019436,3,71.0,60.0,66.0,124.0,108.0,122.0,1,0.216872,-0.055456,...,-0.466976,-0.189028,0.048556,-0.476408,0.064047,-0.008339,0.424513,0.450524,-0.022595,0.231871
1043241,0,40.0,40.0,43.0,128.0,106.0,120.0,1,-0.060757,0.218841,...,-0.118490,-0.123154,-0.287799,-0.404065,0.111559,-0.233413,-0.120683,0.083335,-0.058685,0.343333
1266183,0,47.0,44.0,43.0,136.0,96.0,120.0,1,-0.063801,0.061519,...,-0.256891,-0.414658,-0.549495,0.543498,0.292311,0.164908,-0.031181,0.084472,-0.235999,-0.609832
1535233,0,42.0,41.0,43.0,106.0,135.0,122.0,1,0.022708,0.380467,...,-0.238173,-0.059068,-0.020537,-0.465405,0.143333,0.194637,0.410916,0.510831,-0.106011,0.137268


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC480,PC481,PC482,PC483,PC484,PC485,PC486,PC487,PC488,DX
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,-19.214669,28.933982,7.124812,-4.864777,-0.918411,1.556838,-33.216430,3.703853,25.597683,1.954574,...,-0.703743,1.431570,0.585098,1.311178,2.560978,1.058323,0.699934,-2.087027,0.238786,0
1019436,-18.864300,-40.754030,-33.519231,13.250311,-0.775394,0.647756,-12.077802,-19.265826,22.666633,-2.840693,...,0.590804,-4.186011,-1.626872,-5.879905,-1.606961,-1.246036,-1.583198,0.242572,2.373836,3
1043241,7.846577,20.948205,-36.247873,20.999480,16.510235,17.077271,-2.285614,-1.724268,14.021748,-16.138031,...,0.754514,-1.506335,1.275221,-0.766122,-4.018175,2.018278,0.826163,-2.245247,1.847750,0
1266183,-8.407607,32.852001,-19.645893,13.501739,-2.173091,4.952939,-10.915426,2.953900,-3.423763,11.618529,...,-1.469141,-2.070394,-0.410683,-1.465609,0.479998,1.876242,-0.523378,1.685966,0.783129,0
1535233,-2.888877,5.990861,-8.160449,30.408660,44.460316,12.253571,-0.593780,9.542005,23.975279,15.386297,...,-1.053952,2.432530,-0.439854,0.909798,1.141703,0.357122,3.979956,1.749881,-4.279552,0


,ScanDir ID,DX,ReHo_1,ReHo_2,ReHo_3,ReHo_4,ReHo_5,ReHo_6,ReHo_7,ReHo_8,...,ReHo_181,ReHo_182,ReHo_183,ReHo_184,ReHo_185,ReHo_186,ReHo_187,ReHo_188,ReHo_189,ReHo_190
Subject ID,,,,,,,,,,,,,,,,,,,,,
0,1018959,0,-0.000693,-0.002784,0.005485,0.013734,0.009531,0.012442,0.015524,0.004018,...,-0.000228,-0.004366,0.007430,0.014627,0.001317,-0.006228,0.001716,-0.000210,-0.012670,-0.006496
1,1019436,3,0.001641,0.000157,-0.000600,0.008507,-0.005215,0.021348,0.037630,-0.002221,...,0.005336,0.026657,-0.010433,0.010363,0.016745,0.020867,-0.026912,-0.010929,0.002073,-0.014229
2,1043241,0,-0.025526,-0.007973,-0.003846,-0.008930,0.002580,0.012220,0.027966,0.006901,...,-0.004057,0.029532,0.017034,0.011103,-0.010745,-0.009088,0.008249,-0.008181,-0.001200,-0.007629
3,1266183,0,0.017338,-0.020910,0.026822,-0.028537,0.004145,0.018429,0.023084,0.018687,...,-0.009916,0.024002,0.001976,-0.016781,0.019819,0.008669,0.015358,-0.016745,-0.011874,0.012493
4,1535233,0,0.025408,-0.004181,0.019778,0.010639,0.042414,0.037103,0.037410,0.053010,...,-0.033017,-0.009099,-0.008167,-0.052603,0.034772,-0.018005,0.037081,0.002573,0.026957,0.030917


,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,DX
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0.440059,0.654038,0.604660,0.539188,0.427841,0.556644,0.456504,0.500982,0.631251,0.613161,...,0.375033,0.613263,0.645600,0.468351,0.381020,0.650361,0.684567,0.676759,0.612947,0
1019436,0.756351,0.527825,0.619720,0.466123,0.497030,0.384478,0.546645,0.465267,0.588696,0.561305,...,0.492485,0.475318,0.603202,0.554377,0.620290,0.637574,0.674873,0.568847,0.609281,3
1043241,0.730804,0.413692,0.591623,0.642186,0.434255,0.555075,0.479140,0.540595,0.621012,0.618000,...,0.584502,0.637769,0.727549,0.557828,0.637839,0.755363,0.547045,0.572328,0.555771,0
1266183,0.324953,0.566304,0.556289,0.538490,0.495830,0.416628,0.328429,0.372241,0.477356,0.531180,...,0.448431,0.631884,0.497536,0.458568,0.515995,0.474847,0.524475,0.535504,0.719011,0
1535233,0.544618,0.434999,0.646133,0.715492,0.669921,0.582833,0.459039,0.543931,0.610697,0.451788,...,0.539778,0.641483,0.671250,0.642956,0.653623,0.727645,0.405513,0.391241,0.657757,0


In [6]:
FC_data['DX'] = FC_data['DX'].apply(lambda x: 1 if x > 0 else 0)
FC_pheno_data['DX'] = FC_pheno_data['DX'].apply(lambda x: 1 if x > 0 else 0)
FC_PCA['DX'] = FC_PCA['DX'].apply(lambda x: 1 if x > 0 else 0)
Reho['DX'] = Reho['DX'].apply(lambda x: 1 if x > 0 else 0)
falff['DX'] = falff['DX'].apply(lambda x: 1 if x > 0 else 0)

FC_data
FC_PCA
FC_pheno_data
Reho
falff

,DX,0,1,2,3,4,5,6,7,8,...,17945,17946,17947,17948,17949,17950,17951,17952,17953,17954
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0,0.082767,-0.202121,-0.253291,0.143162,-0.212533,0.572503,-0.346531,0.010921,0.014705,...,0.112687,0.171248,-0.073971,-0.413591,0.238855,0.300857,0.177778,0.658069,-0.052805,0.145006
1019436,1,0.216872,-0.055456,0.274632,0.057173,0.318357,0.334924,-0.285290,-0.093749,0.051842,...,-0.466976,-0.189028,0.048556,-0.476408,0.064047,-0.008339,0.424513,0.450524,-0.022595,0.231871
1043241,0,-0.060757,0.218841,-0.220541,-0.009787,-0.018797,0.102055,-0.207456,-0.332605,0.157679,...,-0.118490,-0.123154,-0.287799,-0.404065,0.111559,-0.233413,-0.120683,0.083335,-0.058685,0.343333
1266183,0,-0.063801,0.061519,-0.011792,0.016329,0.135984,0.164675,0.174189,-0.108558,-0.031854,...,-0.256891,-0.414658,-0.549495,0.543498,0.292311,0.164908,-0.031181,0.084472,-0.235999,-0.609832
1535233,0,0.022708,0.380467,0.404897,0.422225,0.546707,-0.022366,0.218834,0.274712,0.191838,...,-0.238173,-0.059068,-0.020537,-0.465405,0.143333,0.194637,0.410916,0.510831,-0.106011,0.137268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669389,0,-0.136454,-0.065583,0.017311,0.052200,0.460014,0.306935,0.176286,-0.345477,0.164890,...,0.206593,0.249444,-0.039177,0.034391,0.666300,0.386416,0.166788,0.555021,0.308751,0.346726
6383713,1,0.004203,-0.314136,-0.223007,0.475450,0.595427,0.529921,-0.154340,-0.193150,-0.176574,...,0.274156,0.141414,-0.183968,-0.383677,0.533130,0.236219,0.114466,0.385278,0.143373,0.421396
6477085,0,-0.619545,-0.075170,-0.074883,0.100222,0.329930,0.071147,-0.110990,0.133600,-0.494886,...,0.137766,0.480185,0.059669,0.074272,0.309700,0.376793,0.156937,0.225204,0.034719,0.042565


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC480,PC481,PC482,PC483,PC484,PC485,PC486,PC487,PC488,DX
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,-19.214669,28.933982,7.124812,-4.864777,-0.918411,1.556838,-33.216430,3.703853,25.597683,1.954574,...,-0.703743,1.431570,0.585098,1.311178,2.560978,1.058323,0.699934,-2.087027,0.238786,0
1019436,-18.864300,-40.754030,-33.519231,13.250311,-0.775394,0.647756,-12.077802,-19.265826,22.666633,-2.840693,...,0.590804,-4.186011,-1.626872,-5.879905,-1.606961,-1.246036,-1.583198,0.242572,2.373836,1
1043241,7.846577,20.948205,-36.247873,20.999480,16.510235,17.077271,-2.285614,-1.724268,14.021748,-16.138031,...,0.754514,-1.506335,1.275221,-0.766122,-4.018175,2.018278,0.826163,-2.245247,1.847750,0
1266183,-8.407607,32.852001,-19.645893,13.501739,-2.173091,4.952939,-10.915426,2.953900,-3.423763,11.618529,...,-1.469141,-2.070394,-0.410683,-1.465609,0.479998,1.876242,-0.523378,1.685966,0.783129,0
1535233,-2.888877,5.990861,-8.160449,30.408660,44.460316,12.253571,-0.593780,9.542005,23.975279,15.386297,...,-1.053952,2.432530,-0.439854,0.909798,1.141703,0.357122,3.979956,1.749881,-4.279552,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669389,-25.192405,23.763625,0.439288,-19.330222,-5.943365,4.132156,13.521360,-3.090829,-8.409153,3.890844,...,1.738851,0.762071,-1.074093,1.240432,1.407384,-0.274875,-2.584719,-2.105952,1.334007,0
6383713,2.900486,14.632304,-1.745459,-9.719283,-0.480417,-2.942394,22.701577,-19.565588,-28.477635,0.433893,...,4.079099,-0.999738,3.894240,0.091074,-0.708024,0.313769,3.343667,-2.401685,-0.421493,1
6477085,13.228229,-2.859517,-12.090537,7.090276,34.539183,-1.278220,28.161854,0.327277,-19.166537,-6.277526,...,2.445118,3.092194,-4.391620,1.781686,3.571165,0.911019,-3.847577,3.476473,1.880725,0


,DX,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status,0,1,...,17945,17946,17947,17948,17949,17950,17951,17952,17953,17954
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0,44.0,47.0,44.0,99.0,115.0,103.0,1,0.082767,-0.202121,...,0.112687,0.171248,-0.073971,-0.413591,0.238855,0.300857,0.177778,0.658069,-0.052805,0.145006
1019436,1,71.0,60.0,66.0,124.0,108.0,122.0,1,0.216872,-0.055456,...,-0.466976,-0.189028,0.048556,-0.476408,0.064047,-0.008339,0.424513,0.450524,-0.022595,0.231871
1043241,0,40.0,40.0,43.0,128.0,106.0,120.0,1,-0.060757,0.218841,...,-0.118490,-0.123154,-0.287799,-0.404065,0.111559,-0.233413,-0.120683,0.083335,-0.058685,0.343333
1266183,0,47.0,44.0,43.0,136.0,96.0,120.0,1,-0.063801,0.061519,...,-0.256891,-0.414658,-0.549495,0.543498,0.292311,0.164908,-0.031181,0.084472,-0.235999,-0.609832
1535233,0,42.0,41.0,43.0,106.0,135.0,122.0,1,0.022708,0.380467,...,-0.238173,-0.059068,-0.020537,-0.465405,0.143333,0.194637,0.410916,0.510831,-0.106011,0.137268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669389,0,24.0,15.0,9.0,120.0,97.0,110.0,1,-0.136454,-0.065583,...,0.206593,0.249444,-0.039177,0.034391,0.666300,0.386416,0.166788,0.555021,0.308751,0.346726
6383713,1,61.0,29.0,32.0,115.0,91.0,104.0,1,0.004203,-0.314136,...,0.274156,0.141414,-0.183968,-0.383677,0.533130,0.236219,0.114466,0.385278,0.143373,0.421396
6477085,0,25.0,13.0,12.0,115.0,112.0,115.0,1,-0.619545,-0.075170,...,0.137766,0.480185,0.059669,0.074272,0.309700,0.376793,0.156937,0.225204,0.034719,0.042565


,ScanDir ID,DX,ReHo_1,ReHo_2,ReHo_3,ReHo_4,ReHo_5,ReHo_6,ReHo_7,ReHo_8,...,ReHo_181,ReHo_182,ReHo_183,ReHo_184,ReHo_185,ReHo_186,ReHo_187,ReHo_188,ReHo_189,ReHo_190
Subject ID,,,,,,,,,,,,,,,,,,,,,
0,1018959,0,-0.000693,-0.002784,0.005485,0.013734,0.009531,0.012442,0.015524,0.004018,...,-0.000228,-0.004366,0.007430,0.014627,0.001317,-0.006228,0.001716,-0.000210,-0.012670,-0.006496
1,1019436,1,0.001641,0.000157,-0.000600,0.008507,-0.005215,0.021348,0.037630,-0.002221,...,0.005336,0.026657,-0.010433,0.010363,0.016745,0.020867,-0.026912,-0.010929,0.002073,-0.014229
2,1043241,0,-0.025526,-0.007973,-0.003846,-0.008930,0.002580,0.012220,0.027966,0.006901,...,-0.004057,0.029532,0.017034,0.011103,-0.010745,-0.009088,0.008249,-0.008181,-0.001200,-0.007629
3,1266183,0,0.017338,-0.020910,0.026822,-0.028537,0.004145,0.018429,0.023084,0.018687,...,-0.009916,0.024002,0.001976,-0.016781,0.019819,0.008669,0.015358,-0.016745,-0.011874,0.012493
4,1535233,0,0.025408,-0.004181,0.019778,0.010639,0.042414,0.037103,0.037410,0.053010,...,-0.033017,-0.009099,-0.008167,-0.052603,0.034772,-0.018005,0.037081,0.002573,0.026957,0.030917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,5669389,0,0.041583,0.031368,0.018500,0.003853,0.005902,0.035916,0.007781,0.037332,...,0.029162,-0.003820,0.023286,-0.027044,0.047881,-0.004888,0.033479,0.043651,0.021794,0.022059
616,6383713,1,-0.011001,0.023553,0.045278,0.023125,-0.010543,0.009812,-0.002366,0.047632,...,0.023663,-0.011299,0.034275,0.012848,0.053413,-0.014896,0.022261,0.042729,0.033602,0.029257
617,6477085,0,-0.090311,0.077690,0.036559,-0.020320,0.005736,-0.015036,-0.008798,0.056555,...,0.046824,0.049709,0.059777,0.007273,0.057467,0.073680,0.053626,0.080742,0.041598,0.056369


,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,DX
Subject ID,,,,,,,,,,,,,,,,,,,,,
1018959,0.440059,0.654038,0.604660,0.539188,0.427841,0.556644,0.456504,0.500982,0.631251,0.613161,...,0.375033,0.613263,0.645600,0.468351,0.381020,0.650361,0.684567,0.676759,0.612947,0
1019436,0.756351,0.527825,0.619720,0.466123,0.497030,0.384478,0.546645,0.465267,0.588696,0.561305,...,0.492485,0.475318,0.603202,0.554377,0.620290,0.637574,0.674873,0.568847,0.609281,1
1043241,0.730804,0.413692,0.591623,0.642186,0.434255,0.555075,0.479140,0.540595,0.621012,0.618000,...,0.584502,0.637769,0.727549,0.557828,0.637839,0.755363,0.547045,0.572328,0.555771,0
1266183,0.324953,0.566304,0.556289,0.538490,0.495830,0.416628,0.328429,0.372241,0.477356,0.531180,...,0.448431,0.631884,0.497536,0.458568,0.515995,0.474847,0.524475,0.535504,0.719011,0
1535233,0.544618,0.434999,0.646133,0.715492,0.669921,0.582833,0.459039,0.543931,0.610697,0.451788,...,0.539778,0.641483,0.671250,0.642956,0.653623,0.727645,0.405513,0.391241,0.657757,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669389,0.256059,0.476082,0.668876,0.450693,0.398631,0.544864,0.429395,0.567268,0.646345,0.525099,...,0.316761,0.561301,0.515058,0.305818,0.312659,0.622423,0.618545,0.537619,0.563718,0
6383713,0.229875,0.445956,0.446567,0.515856,0.507962,0.443237,0.339974,0.519716,0.503483,0.485144,...,0.332223,0.643870,0.490933,0.395509,0.346755,0.377433,0.524141,0.334546,0.418346,1
6477085,0.340038,0.422289,0.542378,0.624692,0.546239,0.549973,0.443172,0.644051,0.535838,0.474425,...,0.383385,0.465567,0.499877,0.352168,0.394593,0.424113,0.480928,0.514339,0.584390,0


## Hyperparameter Tuning

In [8]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter search space
param_dist = {
    'n_estimators': randint(80, 120),
    'learning_rate': uniform(0.05, 0.1),
    'max_depth': randint(3, 6),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2),
    'gamma': uniform(0, 0.3),
    'lambda': uniform(1, 3)
}

results = []

### Functional Connectivity Only

In [10]:
# Define your X (features) and y (labels) before this
X = FC_data.drop(columns = "DX")
y = FC_data["DX"]

# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'Functional Connectivity',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})



Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4D9900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D9D4DABF0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D95059FC0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4DAEC0>},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4D9900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D9D4DABF0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D95059FC0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4DAEC0>},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4D9900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D9D4DABF0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D95059FC0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4DAEC0>},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4D9900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D9D4DABF0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D95059FC0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4DAEC0>},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4D9900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D9D4DABF0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022D95059FC0>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000022D9D4DAEC0>},
                   random_state=42, scoring='f1', verbose=1)

In [11]:
results

[{'Dataset': 'Functional Connectivity',
  'CV F1': 0.51766,
  'Final Classification Report': {'0': {'precision': 0.6178010471204188,
    'recall': 0.6941176470588235,
    'f1-score': 0.6537396121883656,
    'support': 340.0},
   '1': {'precision': 0.5630252100840336,
    'recall': 0.4785714285714286,
    'f1-score': 0.5173745173745173,
    'support': 280.0},
   'accuracy': 0.5967741935483871,
   'macro avg': {'precision': 0.5904131286022263,
    'recall': 0.5863445378151261,
    'f1-score': 0.5855570647814414,
    'support': 620.0},
   'weighted avg': {'precision': 0.5930635723297932,
    'recall': 0.5967741935483871,
    'f1-score': 0.5921553758208212,
    'support': 620.0}}}]

In [12]:
# Define your X (features) and y (labels) before this
X = FC_pheno_data.drop(columns="DX")
y = FC_pheno_data["DX"]

param_dist = {
    'n_estimators': [80, 100, 120, 150, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'lambda': [1, 2]
}


# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'Functional Connectivity with pheno',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})




Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)

### Functional Connectivity PCA

In [14]:
# Define your X (features) and y (labels) before this
X = FC_PCA.drop(columns="DX")
y = FC_PCA["DX"]

param_dist = {
    'n_estimators': [80, 100, 120, 150, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'lambda': [1, 2]
}


# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'Functional Connectivity PCA',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})






Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)

## ReHo

In [16]:
# Define your X (features) and y (labels) before this
X = Reho.drop(columns=["DX", "ScanDir ID"])
y = Reho["DX"]

param_dist = {
    'n_estimators': [80, 100, 120, 150, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'lambda': [1, 2]
}


# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'ReHo',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})






Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)

### fALFF

In [18]:
# Define your X (features) and y (labels) before this
X = falff.drop(columns="DX")
y = falff["DX"]

param_dist = {
    'n_estimators': [80, 100, 120, 150, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'lambda': [1, 2]
}


# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'Falff',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})






Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)

## Combined (PCA + ReHo + fALFF + pheno)

In [20]:
# Step 1: Define phenotype columns to include
phenotype_cols = ['Inattentive', 'Hyper/Impulsive', 'Verbal IQ', 
                  'Performance IQ', 'Full4 IQ', 'Med Status']

# Extract phenotype subset from FC_pheno_data
pheno_data = FC_pheno_data[phenotype_cols + ['DX']].copy()
pheno_data.index.name = 'Subject ID'

# Step 2: Align all features by Subject ID
Reho_indexed = Reho.set_index('ScanDir ID')
fALFF_indexed = falff.copy()
PCA_indexed = FC_PCA.copy()

# Step 3: Get intersection of indices
common_subjects = list(set(PCA_indexed.index) & set(Reho_indexed.index) & set(fALFF_indexed.index) & set(pheno_data.index))

# Step 4: Subset all datasets to common subjects
Reho_sub = Reho_indexed.loc[common_subjects]
falff_sub = fALFF_indexed.loc[common_subjects]
pca_sub = PCA_indexed.loc[common_subjects]
pheno_sub = pheno_data.loc[common_subjects]

# Step 5: Concatenate datasets
merged_df = pd.concat([pca_sub, Reho_sub, falff_sub, pheno_sub], axis=1)

# Drop duplicate DX columns if they exist
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Display merged DataFrame

merged_df.shape


(493, 875)

In [21]:
# Define your X (features) and y (labels) before this
# Then define X and y
X = merged_df.drop(columns="DX")
y = merged_df["DX"]


param_dist = {
    'n_estimators': [80, 100, 120, 150, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0, 0.2],
    'lambda': [1, 2]
}


# To track predictions and scores
y_true_all = []
y_pred_all = []
f1_scores = []
acc_scores = []
best_params_list = []

# CV loop
for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold_idx + 1} of 5")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs= 1
    )

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=15,
        scoring='f1',
        cv=2,
        verbose=1,
        random_state=42,
        n_jobs= 1
    )

    search.fit(X_train, y_train)
    
    best_model = search.best_estimator_

    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Track overall predictions and metrics
    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)
    f1_scores.append(f1)
    acc_scores.append(acc)
    best_params_list.append(search.best_params_)

# Final classification report
final_report = classification_report(y_true_all, y_pred_all, output_dict=True)

results.append({
    'Dataset': 'Combined',  
    'CV F1': round(np.mean(f1_scores), 6),
    'Final Classification Report': final_report
})






Fold 1 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 2 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 3 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 4 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)


Fold 5 of 5
Fitting 2 folds for each of 15 candidates, totalling 30 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           gpu_id=0, grow_policy=None,
                                           importance_type=None,
                                           interac...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=1, ...),
                   n_iter=15, n_jobs=1,
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0, 0.2], 'lambda': [1, 2],
                                        'learning_rate': [0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6],
                                        'n_estimators': [80, 100, 120, 150,
                                                         200],
                                        'subsample': [0.8]},
                   random_state=42, scoring='f1', verbose=1)

In [22]:
results
results_df = pd.DataFrame(results)
results_df

[{'Dataset': 'Functional Connectivity',
  'CV F1': 0.51766,
  'Final Classification Report': {'0': {'precision': 0.6178010471204188,
    'recall': 0.6941176470588235,
    'f1-score': 0.6537396121883656,
    'support': 340.0},
   '1': {'precision': 0.5630252100840336,
    'recall': 0.4785714285714286,
    'f1-score': 0.5173745173745173,
    'support': 280.0},
   'accuracy': 0.5967741935483871,
   'macro avg': {'precision': 0.5904131286022263,
    'recall': 0.5863445378151261,
    'f1-score': 0.5855570647814414,
    'support': 620.0},
   'weighted avg': {'precision': 0.5930635723297932,
    'recall': 0.5967741935483871,
    'f1-score': 0.5921553758208212,
    'support': 620.0}}},
 {'Dataset': 'Functional Connectivity with pheno',
  'CV F1': 0.912089,
  'Final Classification Report': {'0': {'precision': 0.8970099667774086,
    'recall': 0.9818181818181818,
    'f1-score': 0.9375,
    'support': 275.0},
   '1': {'precision': 0.9739583333333334,
    'recall': 0.8577981651376146,
    'f1-sco

,Dataset,CV F1,Final Classification Report
0,Functional Connectivity,0.517660,"{'0': {'precision': 0.6178010471204188, 'recal..."
1,Functional Connectivity with pheno,0.912089,"{'0': {'precision': 0.8970099667774086, 'recal..."
2,Functional Connectivity PCA,0.487868,"{'0': {'precision': 0.6020408163265306, 'recal..."
3,ReHo,0.463536,"{'0': {'precision': 0.5871794871794872, 'recal..."
4,Falff,0.503135,"{'0': {'precision': 0.6122448979591837, 'recal..."
5,Combined,0.944154,"{'0': {'precision': 0.9530685920577617, 'recal..."


In [39]:
final_report = classification_report(y_true_all, y_pred_all)

In [41]:
final_report

'              precision    recall  f1-score   support\n\n           0       0.95      0.96      0.96       275\n           1       0.95      0.94      0.94       218\n\n    accuracy                           0.95       493\n   macro avg       0.95      0.95      0.95       493\nweighted avg       0.95      0.95      0.95       493\n'

In [48]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data = cm, 
            columns = pd.MultiIndex.from_tuples([("Model Predicted", "Control"), ("Model Predicted", "ADHD")]),
            index = pd.MultiIndex.from_tuples([("Actual Values", "Control"),("Actual Values", "ADHD")]))                    

Model Predicted     
                              Control ADHD
Actual Values Control              52    3
              ADHD                  2   41